<a href="https://colab.research.google.com/github/relfarizi/Final_Project/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Library

In [ ]:
# Install rdkit

def install_rdkit():
  !wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
  !chmod +x Miniconda3-latest-Linux-x86_64.sh
  !time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
  !time conda install -q -y -c conda-forge rdkit

# install_rdkit()

In [116]:
# Data

#from google.colab import files
import pandas as pd
import numpy as np
import regex as re

# rdkit
import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages/')
from rdkit import Chem
from rdkit.Chem import DataStructs

# random
import random as rd

## Data

### Pre-Processing Data

In [ ]:
def preprocessing_data(link_of_data) :
  dataset = pd.read_csv(link_of_data,sep=";")
  dataset = dataset.iloc[:,[0,7,9,10,11]]

  list_of_column_dataframe = dataset.columns
  list_of_dataset_after = []

  list_of_dataset_before = np.array(dataset)
  
  for data in list_of_dataset_before :
    if data[2] == "'='" :
      if data[4] == "nM" :
        if data[3] < 10000 :
          data[3] = data[3]/1000
          list_of_dataset_after.append(data)
  
  dataframe_of_dataset_after = pd.DataFrame(list_of_dataset_after)
  dataframe_of_dataset_after.columns = list_of_column_dataframe
  name_of_file_download = "Dataset_After.csv"
  dataframe_of_dataset_after.to_csv(name_of_file_download)
  files.download(name_of_file_download)
  return dataframe_of_dataset_after


In [ ]:
#preprocessing_data("https://raw.githubusercontent.com/relfarizi/Final_Project/master/Data/Dataset_Before.csv")

### Data After Pre-Processing

In [ ]:
# for colab

#dataset = pd.read_csv('https://raw.githubusercontent.com/relfarizi/Final_Project/master/Data/Dataset_After.csv', index_col= 0)


dataset = pd.read_csv('Data/Dataset_After.csv',index_col = 0)
dataset.head(5)

In [ ]:
#dataset = dataset.drop_duplicates(keep=False,inplace=True)
dataset = dataset.dropna()
mol_name = dataset.iloc[:,0].values.tolist()
row_drop = []
for i in range(len(mol_smiles)) :
  #print(mol_smiles[i])
  if len(str(mol_smiles[i])) < 5 :
    row_drop.append(i)
print(row_drop)
dataset.drop(row_drop)
dataset.iloc[258,:]

## Probability Activity

### 1. Molecule Activity
Untuk mencari jumlah dari nilai aktivitas masing2 molekul

variabel yang digunakan :
* mol_name = list nama molekul
* mol_smile = list smile molekul
* mol_act = list berisi nilai IC50
* act = nilai aktivitas tiap molekul
* sum_act = Nilai jumlah dari a

In [ ]:
mol_name = dataset.iloc[:,0].values.tolist()
mol_smiles = dataset.iloc[:,1].values.tolist()
mol_act = dataset.iloc[:,3].values.tolist()
sum_act = 0
for molekul in mol_act :
  act = np.exp(-molekul)
  sum_act  += act
print(sum_act)

### 2. Probability Succes


Mencari nilai probabilitas kesuksesan untuk setiap molekul

Variabel yang digunakan :
* n = banyaknya molekul
* mol_act_dict = Dictioary yang akan menampung nilai activity dan probabilitas
>* mol_act_dict[keys][0] = IC50(dari data)
>* mol_act_dict[keys][1] = Aktifitas Molekul (dari rumus)
>* mol_act_dict[keys][2] = Probabilitas Kesuksesan
* act = nilai aktivitas molekul
* prob = nilai probabilitas kesuksesan molekul
* sum_pro = nilai jumlah dari Probabiliras Kesuksesan




In [ ]:
n = len(mol_act)
#p_avg = 1/n
mol_prob = []

# act = probabilitas kesuksesan

sum_pro = 0
for molekul in mol_act :
  #act = (molekul[1]*p_avg*n)/sum_act
  act = np.exp(-molekul)
  prob = act/sum_act
  prob = round(prob,8)
  mol_prob.append(prob)
  sum_pro += prob
print(sum_pro)


## Fingerprint

In [151]:
mol_fp = []
i = 0
print(mol_smiles[258])
for molekul in mol_smiles :
  smiles = Chem.MolFromSmiles(str(molekul))
  fp = Chem.RDKFingerprint(smiles)
  #print(i)
  i+= 1
  mol_fp.append(fp)
print((mol_fp))

COc1ccc(OC)c([C@@H]2O[C@H](CNC(=O)c3ccccc3)[C@@H](O)[C@H](OC(=O)c3ccccc3)[C@H]2OC(=O)c2ccccc2)c1
[<rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE0115F80>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE0283BC0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE02837B0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE02DB300>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE01CD9E0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE02838F0>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE02D7F80>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE02D7F30>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE02D7B70>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE02D7D50>, <rdkit.DataStructs.cDataStructs.ExplicitBitVect object at 0x0000026EE02D7DA0>, <rdkit.DataStructs.cDataStructs.E

## To DataFrame

In [241]:
#df_mol = pd.DataFrame.from_dict(mol_act_dict, orient='index')
#df_mol.columns = ["IC50","Molecule Activity", "Probability Activity"]
#df_mol.sort_values(by=["IC50"], ascending=False)
#mol_name = pd.Series(mol_name)
#mol_act = pd.Series(mol_act)
#df_mol = pd.concat(mol_name,mol_act)
#print(len(mol_name))
#print(len(mol_smiles))
#print(len(mol_fp))
#print(len(mol_act))
#print(len(mol_prob))

df_mol = pd.DataFrame({'mol_name':mol_name,
                       'mol_smiles' : mol_smiles,
                       'mol_act':mol_act,
                       'mol_fp' : mol_fp,
                       'mol_prob' : mol_prob
                       })

df_mol.head()

,mol_name,mol_smiles,mol_act,mol_fp,mol_prob
0,CHEMBL2263120,CC1(C)CC[C@]2(C(=O)OCCCCCCCCCCC(=O)O)CC[C@]3(C...,0.7200,"[0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",0.000790
1,CHEMBL2263121,CC1(C)CC[C@]2(C(=O)NC(Cc3ccccc3)C(=O)O)CC[C@]3...,3.3100,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, ...",0.000059
2,CHEMBL168,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,5.0501,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",0.000010
3,CHEMBL508831,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,0.5400,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...",0.000946
4,CHEMBL506556,CC1(C)CC[C@]2(C(=O)O)CC[C@]3(C)C(=CC[C@@H]4[C@...,0.6200,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, ...",0.000873


In [242]:
df_mol = df_mol.iloc[:,[0,2,3]]
df_mol

,mol_name,mol_act,mol_fp
0,CHEMBL2263120,0.7200,"[0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ..."
1,CHEMBL2263121,3.3100,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, ..."
2,CHEMBL168,5.0501,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ..."
3,CHEMBL508831,0.5400,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ..."
4,CHEMBL506556,0.6200,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, ..."
...,...,...,...
1847,CHEMBL1782241,5.2000,"[0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, ..."
1848,CHEMBL323197,5.5000,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, ..."
1849,CHEMBL4071698,0.7200,"[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1850,CHEMBL3770020,1.5800,"[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, ..."


## Individu

In [245]:
n_subset = 5

def individu(subset):
    subset = np.array(subset)
    mol_name = subset[:,0]
    mol_act = subset[:,1]
    mol_fp = subset[:,2]
    print(mol_name)
    print(mol_act)
    print(mol_fp)

    
    fp_list = []
    n = len(subset)
    
    for i in range(n):
        for j in range(n):
            if i > j :
                tmp_fp = DataStructs.FingerprintSimilarity(mol_fp[i],mol_fp[j])
                fp_list.append(tmp)
                
    avg_act = np.mean(mol_act)
    avg_div = np.mean(fp_list)
    return [mol_name,avg_act,avg_div]



In [246]:
subset_row = []
sum_of_subset = 10

for i in range(sum_of_subset):
    random = rd.randint(0,len(df_mol))
    while random in subset_row:
        random = rd.randint(0,len(df_mol))
    subset_row.append(random)
print(subset_row)
calon_individu = df_mol.iloc[subset_row,:].values.tolist()
calon_individu

individu(calon_individu)

[1076, 1749, 331, 1071, 919, 1530, 1130, 1154, 1423, 282]
['CHEMBL1801407' 'CHEMBL590235' 'CHEMBL426373' 'CHEMBL2208254'
 'CHEMBL1085070' 'CHEMBL3234648' 'CHEMBL267048' 'CHEMBL1164299'
 'CHEMBL3356399' 'CHEMBL504397']
[4.0 0.005 4.5 5.23 0.10965 1.03 0.12 2.75 3.8 2.72]


[array(['CHEMBL1801407', 'CHEMBL590235', 'CHEMBL426373', 'CHEMBL2208254',
        'CHEMBL1085070', 'CHEMBL3234648', 'CHEMBL267048', 'CHEMBL1164299',
        'CHEMBL3356399', 'CHEMBL504397'], dtype=object),
 2.426465,
 0.8962406015037598]

## For git (Can't be use yet)

file > save a copy in Github

In [100]:
#!git init

#!git config --global user.name “relfarizi”

#!git remote add origin https://github.com/relfarizi/Final_Project.git

#!git add .

#!git commit -m "Probability Activity"

#!git push

#!git status